In [1]:
import sys
# Delete all global variables when re-running the notebook.
this = sys.modules[__name__] # type: ignore
for n in dir():
    if n in ['this', 'was_mounted']: continue
    if n[0]!='_': delattr(this, n)


try:
    was_mounted = was_mounted
except:
    was_mounted = False


import os
if  os.getenv("COLAB_RELEASE_TAG"):
  is_running_on_colab = True

else:
  is_running_on_colab = False

if is_running_on_colab:
  packages_to_install = ['pandas==2.1.3','neurokit2', 'wfdb']

  for package in packages_to_install:
    os.system(f'pip install {package}')
  from google.colab import drive, files
  code_directory = './gdrive/MyDrive/TCC/ectopic_beats_detection'
  if not was_mounted:
      drive.mount('/content/gdrive')
  was_mounted = True
  if not os.path.samefile(os.getcwd(),code_directory):
    os.chdir(code_directory)

from utils import create_compare_df, create_dict_results, plot_results, calculate_metrics, resolve_relative_path
from globals import *
import sys
import neurokit2 as nk
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import pyarrow as pa
from os.path import join
import pyarrow as pa
from typing import Any, Dict, Iterable, List, Tuple, Union
from numpy import typing as npt
from utils import Processor, Processors, load_df_multi_analysis, load_record, apply_processors, correct_peaks
from datetime import datetime
import pickle
from multiprocessing import Pool, cpu_count
from timeit import default_timer as timer
import glob

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 40)

df_record_lead_ann = pd.read_parquet(join(dataframes_directory, 'df_record_lead_ann.parquet'))
df_lead_ann_summery =  pd.read_parquet(join(dataframes_directory, 'df_lead_ann_summery.parquet'))
df_ann_summery = pd.read_parquet(join(dataframes_directory, 'df_ann_summery.parquet'))
df_code_description = pd.read_parquet(join(dataframes_directory, 'df_code_description.parquet'))

df_multi_analysis = load_df_multi_analysis(glob.glob(join(dataframes_directory, 'dict_multi_analysis*.pickle')))

In [2]:
df_multi_analysis[df_multi_analysis[['record_num','processor','method']].duplicated()]

,record_num,processor,method,accuracy,precision,signal_track,start_samples,end_samples


In [3]:
df_multi_analysis.groupby(['record_num']).record_num.count().sort_values().head(5)

record_num
207    27
100    30
205    30
208    30
209    30
Name: record_num, dtype: int64

In [39]:
was_interrupted = False
total_time = 300 # seconds

offset = 100 # seconds
discard_start_sec = discard_end_sec = 2

methods = ['neurokit', 'pantompkins1985', 'hamilton2002', 'martinez2004', 'christov2004',
               'gamboa2008', 'elgendi2010', 'engzeemod2012', 'kalidas2017', 'rodrigues2020']
offset = 500 # seconds

derised_anns = LIST_BEATS_1

try:
    for idx, row in df_record_lead_ann.iterrows():
        if row['upper_signal'] == 'MLII':
            signal_track = 0
        elif row['lower_signal'] == 'MLII':
            signal_track = 1
        else:
            continue

        record_num = row['record'] + 128

        # Start measuring time
        start_time = timer()

        print(f'Processing record {record_num}')

        # Load record
        record, ann = load_record(record_num)
        fs = int(record.fs)  # type: ignore

        samples = int(total_time * fs)
        start_samples = int(offset * fs)
        end_samples = start_samples + samples
        first_used_sample = start_samples + discard_start_sec * fs
        last_used_sample = end_samples - discard_end_sec * fs

        ecg = record.p_signal[:, signal_track][start_samples:end_samples]  # type: ignore
        ecg = pd.Series(ecg, dtype=ECG_TYPE)
        ecg.index += start_samples

        ann_beat_indexes = pd.Series(ann.sample, dtype=INDEX_TYPE)
        ann_beat_symbols = pd.Series(ann.symbol, dtype=ANN_TYPE)

        # Mask for time window and derised annotations
        mask_derised_ann = ann_beat_symbols.isin(derised_anns)

        # We are only interested in samples in the time window
        mask_time_window = (ann_beat_indexes >= start_samples) & (
            ann_beat_indexes < end_samples)
        mask_used_ann = mask_time_window & mask_derised_ann

        # Apply mask
        ann_beat_indexes = ann_beat_indexes[mask_used_ann].reset_index(drop=True)
        ann_beat_symbols = ann_beat_symbols[mask_used_ann].reset_index(drop=True)

        df_beats = correct_peaks(ecg, ann_beat_indexes, fs)

        df_beats = df_beats.rename(columns={'index': 'peak_index', 'local_max': 'cor_peak_index'}).merge(
            pd.DataFrame({'peak_index': ann_beat_indexes, 'symbol': ann_beat_symbols}), on='peak_index', how='left', validate='one_to_one')

        #df_beats.cor_peak_index = df_beats.peak_index

        # If the peak is not corrected, use the original peak index
        df_beats.loc[df_beats.cor_peak_index.isna(
        ), 'cor_peak_index'] = df_beats.peak_index

        raise
except RuntimeError:
    pass


Processing record 228


In [5]:
method = 'hamilton2002'

peaks = dict_results = create_dict_results(ecg, method,start_samples, first_used_sample, last_used_sample, fs, discard_start_sec, discard_end_sec)[method]['local_max'].to_numpy()

In [6]:
# Treat numpy warnings as errors
np.seterr(all='raise')

# By default, classify all beats as normal
classifcations = np.ones(peaks.shape[0], dtype=np.int8)

# Ventricular fibrillation (VF)
vf_episode = False
vf_quant = 0


rr = np.diff(peaks) / fs

for i in range(1,peaks.shape[0] - 3):
    rr1_idx = i; rr1 = rr[rr1_idx]
    rr2_idx = i + 1; rr2 = rr[rr2_idx]
    rr3_idx = i + 2; rr3 = rr[rr3_idx]

    if not vf_episode:
        if (rr2 < 0.6) and (1.8*rr2 < rr1) :
            vf_episode = True
            vf_quant = 1
            classifcations[rr2_idx] = 3
        
        elif (
            ((1.15*rr2 < rr1) and (1.15*rr2 < rr3)) 
            or ((abs(rr1 - rr2) < (0.3)) and (max(rr1,rr2) < (0.8)) and (rr3 > 1.2*((rr1 + rr2)/2)))
            or ((abs(rr2 - rr3) < (0.3)) and (max(rr2,rr3) < (0.8)) and (rr1 > 1.2*((rr2 + rr3)/2))) 
              ):
            classifcations[rr2_idx] = 2
        elif(
            ((2.2) < rr2 < (3.0))
            and ((abs(rr1 - rr2) < (0.2)) or (abs(rr2 - rr3)) < (0.2))
        ):
            classifcations[rr2_idx] = 4
            raise
    else:
        if (max(rr1, rr2, rr3) < (0.7)) or ((rr1 + rr2 + rr3) < (1.7)):
            classifcations[rr2_idx] = 3  # Classify beat as VF
            vf_quant += 1
        else:
            classifcations[rr2_idx] = 1  # Unecessary, but just to be clear
            vf_episode = False

            # If the episode was too short, classify all beats as normal
            if vf_quant < 4:
                for i in range(vf_quant):
                    classifcations[rr2_idx - i] = 1